In [1]:
def get_third_friday(year, month):
    for day in range(15, 22):
        day = datetime.datetime(year, month, day)
        if day.weekday() == 4:
            return day

# determines the front running contract for a given date
def date_to_contract(contract_date):
    third_friday = get_third_friday(contract_date.year, contract_date.month)
    if contract_date.month % 3 == 0:
        if contract_date > third_friday.date():
            if contract_date.month == 12:
                year = contract_date.year + 1
                month = 3
            else:
                year = contract_date.year
                month = contract_date.month + 3
        else:
            year = contract_date.year
            month = contract_date.month
    else:
        year = contract_date.year
        month = (int(contract_date.month / 3) + 1) * 3
    return year, month

In [2]:
from ibapi import wrapper
from ibapi import client
from ibapi import contract
from collections import defaultdict
import os.path as opath
import datetime
import copy

def app_control(func):
    def wrapper(*args, **kwargs):
        args[0]._App__close_application = kwargs['close_app']
        func(*args, **kwargs)
    return wrapper

def app_respond(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        if args[1] in args[0]._App__proc_req_ids:
            args[0]._App__proc_req_ids.remove(args[1])
        if (args[0]._App__close_application == True) and (len(args[0]._App__proc_req_ids) == 0):
            super(App, args[0]).disconnect()
    return wrapper

class Wrapper(wrapper.EWrapper):
    def __init__(self):
        pass

class Client(client.EClient):
    def __init__(self, wrapper):
        client.EClient.__init__(self, wrapper)

class App(Wrapper, Client):
    def __init__(self):
        Wrapper.__init__(self)
        Client.__init__(self, wrapper=self)
        self.historical_data = defaultdict(list)
        self.__current_req_id = None
        self.accountsList = None
        self.__close_application = False
        self.__proc_req_ids = []
        
    def error(self, reqId: int, errorCode: int, errorString: str):
        super().error(reqId, errorCode, errorString)
        print("Error. Id:", reqId, "Code:", errorCode, "Msg:", errorString)

    def historicalData(self, reqId, bar):
        #print("HistoricalData. ReqId:", reqId, "BarData.", bar)
        self.historical_data[reqId].append(bar)
        
    @app_respond
    def historicalDataEnd(self, reqId: int, start: str, end: str):
        super().historicalDataEnd(reqId, start, end)
        print("HistoricalDataEnd. ReqId:", reqId, "from", start, "to", end)
        
    def get_contract(self, symbol, secType='STK', currency='USD', exchange='SMART', futures_month=None):
        sec_contract = contract.Contract()
        sec_contract.includeExpired = True
        sec_contract.symbol = symbol
        sec_contract.secType = secType
        sec_contract.currency = currency
        sec_contract.exchange = exchange
        sec_contract.lastTradeDateOrContractMonth = futures_month
        print(sec_contract.symbol, sec_contract.secType, sec_contract.exchange, sec_contract.currency,
              sec_contract.lastTradeDateOrContractMonth)
        return sec_contract
    
    def get_unique_id(self, filepath='counter.txt'):
        counter = 1
        if not opath.exists(filepath):
            with open(filepath, 'w') as cnt_file:
                cnt_file.write('1')
        else:
            with open(filepath, 'r') as cnt_file:
                counter = int(cnt_file.read())
            with open(filepath, 'w') as cnt_file:
                cnt_file.write(str(counter + 1))
        return counter
    
    @app_control
    def get_historical_data(self, symbol, secType='STK', exchange='SMART', futures_month='',
                            history_len=5, history_unit='D', bar_unit='min', bar_length=1,
                            only_RTH=0, **kwargs):
        unq_id = self.get_unique_id()
        req_id = int(f'{futures_month}{unq_id}')
        self.__current_req_id = req_id
        sec_contract = self.get_contract(symbol, secType=secType, exchange=exchange, futures_month=futures_month)
        sym_data = self.reqHistoricalData(req_id, sec_contract, "", f"{history_len} {history_unit}",
                                          f"{bar_length} {bar_unit}", 'TRADES', only_RTH, 1, False, [])
        self.__proc_req_ids = [req_id]
        return req_id

    @app_control
    def get_historical_data_futures(self, symbol, exchange='GLOBEX',
                            history_len=5, history_unit='D', bar_unit='min', bar_length=1,
                            only_RTH=0, **kwargs):
        days = [(datetime.datetime.now() - datetime.timedelta(days=day)).date() for day in range(history_len)]
        contracts = set([date_to_contract(day) for day in days])
        req_ids = []
        for contract in contracts:
            unq_id = self.get_unique_id()
            req_id = int(f'{contract[0]}{contract[1]:02}{unq_id}')
            self.__current_req_id = req_id
            sec_contract = self.get_contract(symbol, secType='FUT', exchange=exchange,
                                             futures_month=f'{contract[0]}{contract[1]:02}')
            sym_data = self.reqHistoricalData(req_id, sec_contract, "", f"{history_len} {history_unit}",
                                              f"{bar_length} {bar_unit}", 'TRADES', only_RTH, 1, False, [])
            req_ids.append(req_id)
        self.__proc_req_ids = copy.deepcopy(req_ids)
        return req_ids
    
    def get_managed_accounts(self):
        self.reqManagedAccts()
        
    def managedAccounts(self, accountsList):
        print("got account list " + accountsList)
        self.accountsList = accountsList
        
    def get_positions(self):
        self.reqPositions()
        
    def position(self, account: str, contract: contract.Contract, position: float,
                 avgCost: float):
        super().position(account, contract, position, avgCost)
        print("Position.", "Account:", account, "Symbol:", contract.sy
              , "SecType:",
              contract.secType, "Currency:", contract.currency,
              "Position:", position, "Avg cost:", avgCost)
    
    def positionEnd(self):
        super().positionEnd()
        print("PositionEnd")
        
    #next step... functions to place orders

In [3]:
app = App()

In [4]:
app.connect('127.0.0.1',  7497, clientId=0)

futures_month=  '202003'
appl_req_ids = app.get_historical_data(
    'MES', close_app=True, only_RTH=0, history_len=90,
    secType='FUT', exchange='GLOBEX', futures_month=futures_month)

In [5]:
appl_req_ids = app.get_historical_data_futures(
    'MES', close_app=True, only_RTH=0, history_len=60,  bar_length=5, bar_unit='mins',
    secType='FUT', exchange='GLOBEX')

MES FUT GLOBEX USD 202009
MES FUT GLOBEX USD 202006


In [6]:
app.run()

ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


got account list DU1809875
Error. Id: -1 Code: 2104 Msg: Market data farm connection is OK:hfarm
Error. Id: -1 Code: 2104 Msg: Market data farm connection is OK:usfarm.nj
Error. Id: -1 Code: 2104 Msg: Market data farm connection is OK:eufarm
Error. Id: -1 Code: 2104 Msg: Market data farm connection is OK:jfarm
Error. Id: -1 Code: 2104 Msg: Market data farm connection is OK:usfuture
Error. Id: -1 Code: 2104 Msg: Market data farm connection is OK:cashfarm
Error. Id: -1 Code: 2104 Msg: Market data farm connection is OK:usfarm
Error. Id: -1 Code: 2106 Msg: HMDS data farm connection is OK:euhmds
Error. Id: -1 Code: 2106 Msg: HMDS data farm connection is OK:fundfarm
Error. Id: -1 Code: 2106 Msg: HMDS data farm connection is OK:ushmds
Error. Id: -1 Code: 2158 Msg: Sec-def data farm connection is OK:secdefnj
HistoricalDataEnd. ReqId: 2020099 from 20200609  12:59:35 to 20200808  12:59:35
HistoricalDataEnd. ReqId: 20200610 from 20200609  12:59:35 to 20200808  12:59:35


In [7]:
import datetime as datetime
import pandas as pd

def convert_bar_to_df_futures(bars_dict):
    bar_data = []
    for key in bars_dict:
        key_str = str(key)
        year = int(key_str[:4])
        month = int(key_str[4:6])
        for bar in bars_dict[key]:
            bar_data.append(
                [datetime.datetime.strptime(
                    bar.date, '%Y%m%d  %H:%M:%S'
                 ),
                 bar.open, bar.high, bar.low, bar.close, bar.volume, year, month])
            
    df = pd.DataFrame(bar_data, columns=['time', 'open', 'high', 'low', 'close', 'volume',
                                         'contract_year', 'contract_month'])
    unique_dates = df.time.dt.date.unique()
    date_dict = {date: date_to_contract(date) for date in unique_dates}
    expiry = df.time.dt.date.map(date_dict)
    expiry_year, expiry_month = zip(*expiry.values)
    df.loc[:, 'expiry_year'] = expiry_year
    df.loc[:, 'expiry_month'] = expiry_month
    df = df.loc[(df.expiry_year == df.contract_year) & (df.expiry_month == df.contract_month)]
    return df


def convert_bar_to_df(bars):
    bar_data = [ 
        [datetime.datetime.strptime(
            bar.date, '%Y%m%d  %H:%M:%S'
         ),
         bar.open, bar.high, bar.low, bar.close, bar.volume]
        for bar in bars
    ]
    df = pd.DataFrame(bar_data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    return df

In [8]:
df = convert_bar_to_df_futures(app.historical_data)

In [19]:
df.to_parquet('../futures_data/MES.parquet')

In [15]:
df.loc[:, 'dividend'] = 0.0
df.loc[:, 'split'] = 1.0
df.loc[:, 'date'] = df.time.astype(str)

In [20]:
df[['date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'split']].to_csv('../futures_data/MES.csv', index=False)